## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [4]:
cd dev/Master/data

/Users/igor.arambasic/dev/master/data


In [2]:
import pandas as pd

In [8]:
%%time 
bc=pd.read_csv('bookings.csv.bz2',sep='^', chunksize=500000, low_memory=False)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print (i, len(all_chunks))
    


0 500000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000000
6 1000000
7 1000000
8 1000000
9 1000000
10 1499993
11 1499993
12 1499993
13 1499993
14 1499993
15 1499993
16 1499993
17 1499993
18 1499993
19 1499993
20 1499993
CPU times: user 7min 9s, sys: 58.5 s, total: 8min 8s
Wall time: 9min 37s


In [10]:
all_chunks.drop_duplicates(inplace=True)

In [11]:
len(all_chunks)

1299993

In [5]:
all_chunks.to_csv('bookings.no_dup.csv', sep='^', index=False)all_chunks.to_csv('bookings.no_dup.csv', sep='^', index=False)

In [6]:
! head bookings.no_dup.csv

act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      
2013-03-05 00:00:00^1A    ^DE      ^a68dd7ae953c8acfb187a1af2dcbe123^1a11ae49fcbf545fd2afc1a24d88d2b7^ea65900e72d71f4626378e2ebd298267^2013-02-22 00:00:00^1708^0^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHRZRH  ^LONZRH  ^CHGB    ^1^LHRZRH         ^VI^T        ^Y        ^2013-03-07 08:50:00^2013-03-07 11:33:37^-1.0^2013.0^3.0^NULL     
2013-03-26 00:00:00^1A    ^US      ^e612b9eeeee6f17f42d9b0d3b79e75ca^7437560d8f276d6d05eeb806d9e7edee^737295a86982c941f1c2da9a46a14043^2013-03-26 00:00:00^135270^0^SAL     ^SAL     ^SV 

In [7]:
%%time 
bc=pd.read_csv('searches.csv.bz2',sep='^', chunksize=1000000, low_memory=False)

all_chunks=pd.DataFrame()

for i, chunk in enumerate(bc):
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    print (i, len(all_chunks))

0 358999
1 718002
2 718002
3 718002
4 718002
5 718002
6 718002
7 718002
8 718002
9 718002
10 718002
11 718002
12 718002
13 718002
14 718002
15 718002
16 718002
17 718002
18 718002
19 718002
20 718003
CPU times: user 14min 50s, sys: 1min 13s, total: 16min 4s
Wall time: 16min 22s


In [8]:
all_chunks.to_csv('searches.no_dup.csv', sep='^', index=False)

## Action Plan 

In [42]:
#parse_dates did not work with searches
#s=pd.read_csv('searches.no_dup.csv', sep='^', usecols=['Date', 'Origin', 'Destination'], parse_dates=['Date'], errors='raise')
s=pd.read_csv('searches.no_dup.csv', sep='^', usecols=['Date', 'Origin', 'Destination'])

In [43]:
#b=pd.read_csv('bookings.no_dup.csv', sep='^', usecols=['act_date           ', 'dep_port', 'arr_port'], parse_dates=['act_date           '])
b=pd.read_csv('bookings.no_dup.csv', sep='^', usecols=['act_date           ', 'dep_port', 'arr_port'])

In [44]:
s.reset_index(inplace=True) #not really needed at the end

In [14]:
b.head()

,act_date,dep_port,arr_port
0,2013-03-05,ZRH,LHR
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO


In [12]:
s.head()

,Date,Origin,Destination
0,2013-01-01,TXL,AUH
1,2013-01-01,ATH,MIL
2,2013-01-01,ICT,SFO
3,2013-01-01,RNB,ARN
4,2013-01-01,OSL,MAD


#### We have seen that we have white space in some columns....

In [45]:
b['arr_port'][0]

'LHR     '

In [46]:
#removing white spaces from colum names
b.columns=b.columns.str.strip()

In [47]:
b.columns

Index(['act_date', 'dep_port', 'arr_port'], dtype='object')

In [48]:
b['arr_port']=b['arr_port'].str.strip()

In [49]:
b['dep_port']=b['dep_port'].str.strip()

In [63]:
b['bookings']=1

In [64]:
b.head()

,act_date,dep_port,arr_port,bookings
0,2013-03-05,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
5,2013-03-20,DEN,LGA,1
7,2013-03-25,NRT,SIN,1


In [65]:
s.head()

,index,Date,Origin,Destination
0,0,2013-01-01,TXL,AUH
1,1,2013-01-01,ATH,MIL
2,2,2013-01-01,ICT,SFO
3,3,2013-01-01,RNB,ARN
4,4,2013-01-01,OSL,MAD


In [72]:
b.drop_duplicates(inplace=True)

In [78]:
b.head()

,act_date,dep_port,arr_port,bookings
0,2013-03-05,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
5,2013-03-20,DEN,LGA,1
7,2013-03-25,NRT,SIN,1


In [79]:
b['act_date']=b['act_date'].str[:10]

In [80]:
s['Date']=s['Date'].str[:10]

In [81]:
s.head()

,index,Date,Origin,Destination
0,0,2013-01-01,TXL,AUH
1,1,2013-01-01,ATH,MIL
2,2,2013-01-01,ICT,SFO
3,3,2013-01-01,RNB,ARN
4,4,2013-01-01,OSL,MAD


In [82]:
s.tail()

,index,Date,Origin,Destination
717998,717998,2013-01-01,TXL,AUH
717999,717999,2013-01-01,NaN,NaN
718000,718000,2013-01-01,ICT,SFO
718001,718001,2013-01-01,FRA,BGW
718002,718002,2013-10-13,VIE,HA


In [83]:
s_b=s.merge(b,
            how='left',
           left_on=['Origin', 'Destination', 'Date'],
           right_on=['dep_port', 'arr_port', 'act_date'])

In [84]:
s_b.tail()

,index,Date,Origin,Destination,act_date,dep_port,arr_port,bookings
717998,717998,2013-01-01,TXL,AUH,NaN,NaN,NaN,NaN
717999,717999,2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN
718000,718000,2013-01-01,ICT,SFO,NaN,NaN,NaN,NaN
718001,718001,2013-01-01,FRA,BGW,NaN,NaN,NaN,NaN
718002,718002,2013-10-13,VIE,HA,NaN,NaN,NaN,NaN


In [85]:
#lets see the searches with bookings
s_b[ s_b['bookings']>0 ].head()

,index,Date,Origin,Destination,act_date,dep_port,arr_port,bookings
23,23,2013-01-01,DUS,IST,2013-01-01,DUS,IST,1.0
27,27,2013-01-01,RUH,JED,2013-01-01,RUH,JED,1.0
40,40,2013-01-01,DMM,MNL,2013-01-01,DMM,MNL,1.0
59,59,2013-01-01,ATL,MIA,2013-01-01,ATL,MIA,1.0
94,94,2013-01-01,DXB,KUL,2013-01-01,DXB,KUL,1.0


In [86]:
s_b['bookings'].fillna(0, inplace=True)

In [88]:
s_b.tail()

,index,Date,Origin,Destination,act_date,dep_port,arr_port,bookings
717998,717998,2013-01-01,TXL,AUH,NaN,NaN,NaN,0.0
717999,717999,2013-01-01,NaN,NaN,NaN,NaN,NaN,0.0
718000,718000,2013-01-01,ICT,SFO,NaN,NaN,NaN,0.0
718001,718001,2013-01-01,FRA,BGW,NaN,NaN,NaN,0.0
718002,718002,2013-10-13,VIE,HA,NaN,NaN,NaN,0.0


In [93]:
#loading the search file with all the columns 
#we will add bookings column to this file
s_original=pd.read_csv('searches.no_dup.csv', sep='^', low_memory=False)

In [95]:
#this should be the line with the error in Date column
s_original.loc[717999]

Date                 2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5c...
Time                                                               NaN
TxnCode                                                            NaN
OfficeID                                                           NaN
Country                                                            NaN
Origin                                                             NaN
Destination                                                        NaN
RoundTrip                                                          NaN
NbSegments                                                         NaN
Seg1Departure                                                      NaN
Seg1Arrival                                                        NaN
Seg1Date                                                           NaN
Seg1Carrier                                                        NaN
Seg1BookingCode                                                    NaN
Seg2De

### Now we have to put 1 to all the booking with Num of Bookings>1 in the clean file

In [100]:
s_b.drop(['Date', 'Origin', 'Destination', 'index', 'act_date', 'dep_port', 'arr_port' ], axis=1, inplace=True)

In [101]:
result=s_original.merge(s_b, 
                        how='left',
                       left_index=True,
                       right_index=True)

In [102]:
result.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,bookings
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0.0


In [103]:
result.to_csv('result.csv', sep='^', index=False)